<a href="https://colab.research.google.com/github/TranPhu1999/Wordle_solver/blob/main/Play_Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
%%time
# !pip install -r /content/drive/MyDrive/Projects/Wordle_solver/requirements.txt
!python /content/drive/MyDrive/Projects/Wordle_solver/main_v1.py | tee /content/drive/MyDrive/Projects/Wordle_solver/logs/8_40_9_v2.txt


inital_guess:  etao
ground_truth_list:  ['present', 'absent', 'present', 'absent']
100% 346/346 [00:15<00:00, 22.67it/s]

inital_guess:  rale
ground_truth_list:  ['present', 'correct', 'absent', 'correct']
100% 23/23 [00:00<00:00, 402.05it/s]

inital_guess:  rafe
ground_truth_list:  ['present', 'correct', 'absent', 'correct']
100% 21/21 [00:00<00:00, 429.47it/s]

inital_guess:  rase
ground_truth_list:  ['present', 'correct', 'absent', 'correct']
100% 19/19 [00:00<00:00, 467.77it/s]

inital_guess:  rape
ground_truth_list:  ['present', 'correct', 'absent', 'correct']
100% 17/17 [00:00<00:00, 526.90it/s]

inital_guess:  rame
ground_truth_list:  ['present', 'correct', 'present', 'correct']
100% 1/1 [00:00<00:00, 2597.09it/s]

inital_guess:  mare
ground_truth_list:  ['correct', 'correct', 'correct', 'correct']
         657569 function calls (657449 primitive calls) in 17.647 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    3

In [10]:
import requests
from tqdm import tqdm as ProgressDisplay
from itertools import product
from math import log
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import numpy as np

# Disable InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [11]:
from english_words import get_english_words_set
web2lowerset = get_english_words_set(['web2'], lower=True, alpha =False) # gcide / web2
web2lowerset = list(web2lowerset)

In [12]:
common_letter_rank = "etaonrishdlfcmugypwbvkjxzq"
ground_truth = {"absent": 0, "present": 1, "correct": 2}

In [13]:
def check_answer(text, size, seed):
  wordle_url =  f"https://wordle.votee.dev:8000/random?guess={text}&seed={seed}&size={size}"
  r = requests.get(wordle_url, verify = False)
  result = [check["result"] for check in r.json()]
  return result

In [14]:
ground_truth_list = check_answer(text = "thefo",seed = 1234, size = 5)
ground_truth_list

['correct', 'correct', 'correct', 'correct', 'absent']

In [15]:
def get_char_counts(text):
  char_counts = dict()
  characters = list(set(text))
  for char in text:
    if char not in char_counts.keys():
      char_counts[char] = 1
      continue
    char_counts[char] += 1
  return char_counts

In [72]:
# 18s
def get_remain_words_list(guess_text: str,
                          ground_truth_list: np.ndarray,
                          all_text: list[str],
                          allowed_words_list: np.ndarray) -> np.ndarray:
  for i, word in enumerate(allowed_words_list):
    if word:
      for j in range(len(ground_truth_list)): # "absent": 0, "present": 1, "correct": 2
        if (ground_truth_list[j] == 1 and guess_text[j] not in all_text[i]) or \
        (ground_truth_list[j] == 0 and guess_text[j] in all_text[i]) or \
        (ground_truth_list[j] == 2 and guess_text[j] != all_text[i][j]):
          allowed_words_list[i] = 0

  return allowed_words_list


In [73]:
%%time
allowed_text_1 = []

for text in web2lowerset:
  if len(text) == 5:
    allowed_text_1.append(text)

ans = get_remain_words_list("tfyao", [2,1,0,0,0],allowed_text_1,np.ones(len(allowed_text_1)))


CPU times: user 97.6 ms, sys: 0 ns, total: 97.6 ms
Wall time: 100 ms


In [74]:
for i in range(len(ans)):
  if ans[i]:
    print(allowed_text_1[i])

theft
thief
truff


In [75]:
# 24s
def get_remain_words_list(text, ground_truth_list, allowed_words_list):

  remain_list = []
  allowed_words_list = set(allowed_words_list)
  for word in allowed_words_list:
    check_condition = True
    for i, ground_truth in enumerate(ground_truth_list):
      if (ground_truth == "present" and text[i] not in word) or \
      (ground_truth == "absent" and text[i] in word) or \
      (ground_truth == "correct" and text[i] != word[i]):
        check_condition = False

    if check_condition == True:
      remain_list.append(word)
  return remain_list

In [76]:
%%time
allowed_text_1 = []

for text in web2lowerset:
  if len(text) == 5:
    allowed_text_1.append(text)

ground_truth_list = ['correct', 'present', 'absent', 'absent', 'absent']
get_remain_words_list("tfyao", ground_truth_list, allowed_text_1)

CPU times: user 93.4 ms, sys: 0 ns, total: 93.4 ms
Wall time: 94.4 ms


['truff', 'thief', 'theft']

In [68]:
def calculate_entropy(text, all_text: list[str], allowed_words_list, text_size):

  possible_comnination = np.array(list(product(*[[0 ,1 ,2]]*text_size)),dtype=int) # "absent": 0, "present": 1, "correct": 2,
  entropy = 0
  for comnination in possible_comnination:
    # print(text)
    # temp = allowed_words_list.copy()
    remain_words = get_remain_words_list(guess_text = text, ground_truth_list = comnination, all_text = all_text, allowed_words_list = allowed_words_list)
    # print("remain_words: ", remain_words.sum())
    if remain_words.sum() > 0:
      px = remain_words.sum() / allowed_words_list.sum()
      information = log(1/px, 2)
      entropy += px*information
  return entropy


In [19]:
def get_best_guess(all_text: list[str], remain_words: np.ndarray, text_size: int) -> float:
  max_entropy_index = -1
  max_entropy = 0
  for i in ProgressDisplay(range(len(remain_words))):
    if remain_words[i] == 1:
      entropy = calculate_entropy(all_text[i], all_text, remain_words, text_size)
      if max_entropy < entropy:
        max_entropy = entropy
        max_entropy_index = i
  return max_entropy_index


In [20]:
def guess_result(inital_guess: str, all_text: list[str], allowed_words_list: np.ndarray, text_size: int, seed: int) -> str:
  if allowed_words_list.sum() == 1:
    return all_text[list(np.where(allowed_words_list == 1))[0][0]]
  # while len(allowed_words_list) > 1 and found_word == False:
  # best_guess = max(dict_guess, key=dict_guess.get)


  ground_truth_list = check_answer(inital_guess, text_size, seed)
  print("\ninital_guess: ",inital_guess)
  print("ground_truth_list: ",ground_truth_list)

  # "absent": 0, "present": 1, "correct": 2
  ground_truth_list = np.array([ground_truth[i] for i in ground_truth_list],dtype=int)

  if ground_truth_list.sum() == int(2*text_size):
    return inital_guess
  else:
    remain_words = get_remain_words_list(guess_text = inital_guess, ground_truth_list = ground_truth_list, all_text = all_text, allowed_words_list = allowed_words_list)
    max_entropy_index = get_best_guess(all_text = all_text, remain_words = remain_words, text_size = text_size)
    best_guess = all_text[max_entropy_index]
    remain_words[max_entropy_index] = 0
    return guess_result(best_guess, all_text, remain_words, text_size, seed)

In [21]:
def wordle_bot(size, seed):
  all_text = []

  for i, text in enumerate(web2lowerset):
    if len(text) == size:
      all_text.append(text)
  allowed_text = np.ones(len(all_text),dtype=int)

  answer = guess_result(inital_guess = common_letter_rank[:size], all_text = all_text, allowed_words_list = allowed_text, text_size = size, seed = seed)
  return answer

In [24]:
%%time
ans = wordle_bot(size = 4, seed = 3214)
ans


inital_guess:  etao
ground_truth_list:  ['present', 'absent', 'present', 'absent']


100%|██████████| 5006/5006 [00:00<00:00, 97331.25it/s]



inital_guess:  bogy
ground_truth_list:  ['absent', 'absent', 'absent', 'absent']


100%|██████████| 5006/5006 [00:00<00:00, 1122157.33it/s]



inital_guess:  bogy
ground_truth_list:  ['absent', 'absent', 'absent', 'absent']


...


inital_guess:  bogy
ground_truth_list:  ['absent', 'absent', 'absent', 'absent']


100%|██████████| 5006/5006 [00:00<00:00, 518372.69it/s]



inital_guess:  bogy
ground_truth_list:  ['absent', 'absent', 'absent', 'absent']


100%|██████████| 5006/5006 [00:00<00:00, 489627.26it/s]


KeyboardInterrupt: ignored

In [23]:
check_answer(ans,4,3214)

NameError: ignored